In [1]:
import csv
import nltk
import numpy as np
import itertools
import timeit

import sys
sys.path.append('../../conversation-analyzer/src')

import RNNTheano
from RNNTheano import RNNTheano
import util.io as mio

In [20]:
import importlib
import RNNTheano
importlib.reload(RNNTheano)
from RNNTheano import RNNTheano

In [2]:
vocabulary_size = 1000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [3]:
print("Reading conversation and load messages")
messages, senders = mio.parseMessagesFromFile("../../conversation-analyzer/src/resources/unittest/test_plotting.txt")

Reading conversation and load messages


In [4]:
# Read the data and append SENTENCE_START and SENTENCE_END tokens
# Split full message text into sentences
sentences = itertools.chain(*[nltk.sent_tokenize(m.text.lower()) for m in messages])
# Append SENTENCE_START and SENTENCE_END
sentences = ["{} {} {}".format(sentence_start_token, x, sentence_end_token) for x in sentences]
print("Parsed {} sentences.".format(len(sentences)))

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found {} unique words tokens.".format(len(word_freq.items())))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size {}.".format(vocabulary_size))
print("The least frequent word in our vocabulary is '{}' and appeared {} times.".format(vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print("\nExample sentence: '{}'".format(sentences[0]))
print("\nExample sentence after Pre-processing: '{}'".format(tokenized_sentences[0]))

# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

Parsed 2445 sentences.
Found 4042 unique words tokens.
Using vocabulary size 1000.
The least frequent word in our vocabulary is 'reference' and appeared 2 times.

Example sentence: 'SENTENCE_START afar prairie overhead under last all master had ? SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'afar', 'prairie', 'UNKNOWN_TOKEN', 'under', 'last', 'all', 'master', 'had', '?', 'SENTENCE_END']'


In [ ]:
np.random.seed(10)
model = RNNTheano(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print(o.shape)
print(o)

predictions = model.predict(X_train[10])
print(predictions.shape)
print(predictions)

print("Expected Loss for random predictions: {}".format(np.log(vocabulary_size)))
print("Actual loss: {}".format(model.calculate_loss(X_train[:1000], y_train[:1000])))

In [6]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.
Gradient check for parameter %s passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter %s passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter %s passed.


In [21]:
np.random.seed(10)
model = RNNTheano(vocabulary_size)
#%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

INFO (theano.gof.compilelock): Refreshing lock C:\Users\IBM_ADMIN\AppData\Local\Theano\compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.4.5-64\lock_dir\lock


KeyboardInterrupt: 

In [9]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = model.train_with_sgd(X_train, y_train, nepoch=5, evaluate_loss_after=1)

2016-08-11 10:32:44: Loss after num_examples_seen=0 epoch=0: 6.908077568797546
2016-08-11 10:33:45: Loss after num_examples_seen=2445 epoch=1: 4.574073072159846
2016-08-11 10:34:52: Loss after num_examples_seen=4890 epoch=2: 4.529997351523457
2016-08-11 10:36:09: Loss after num_examples_seen=7335 epoch=3: 4.505689009375079
2016-08-11 10:37:19: Loss after num_examples_seen=9780 epoch=4: 4.491194958410326


In [ ]:
num_sentences = 10
senten_min_length = 7

def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs, _ = model.forward_propagation(new_sentence)
        #print(next_word_probs)
        #print( np.array(next_word_probs).shape)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print(" ".join(sent))
    print("###########")